In [1]:
import mesh.parser as mp
import numpy as np
from mesh.npmesh import NPMesh

from scipy import sparse as sps
from scipy.sparse import linalg as splinalg
from scipy import optimize as opt

from matplotlib import pyplot as plt 

In [2]:
with open('../npyfem/resources/paavo_electrostatic_heat.msh') as fileh:
    (nodes, elements) = mp.parse_mesh(fileh)
msh = NPMesh(nodes, elements) #something is heavy here with fine meshes

In [3]:
ele=msh.domains[1001][2]
south=msh.domains[2001][1]
north=msh.domains[2002][1]

Stiffness matrix in weak formulation is
$$ K=\int_\Omega <\sigma \nabla Ni,\nabla N_j> $$
where $$\sigma = \frac{1}{\rho(T)}=\frac{1}{\rho_0(1+\alpha(T-T_1))}
=\frac{1}{\rho_0(1+\alpha((T_1-\frac{r}{\sqrt{2}d}(T_2-T_1))-T_r))}.$$
The constants are $\rho_0=$, $\alpha=$, $d=0.08$m, $T_r=20^\circ$C, $T_1=90^\circ$C and $T_2=390^\circ$C. Naturally $r=\sqrt{x^2+y^2}$.

Furthermore we set $u=u_1$ in $\Gamma_{south}$ and $u=u_2$ in $\Gamma_{north}$.

Defining the conductivity first.

In [4]:
rho0=1
alpha=0
d=0.08
Tr=20
T1=90
T2=390
u1=0
u2=1
#can python/numpy handle function compositions? Would prevent chaos like this:
rho_sc=lambda x: 1/(rho0*(1+alpha*((T1-np.sqrt(x[:,:,:,0]**2+x[:,:,:,1]**2)/(np.sqrt(2)*d)*(T2-T1))-Tr)))

Next, we define integration, shape functions, their derivatives, jacobians and their determinants and the scalar function $\rho$.

In [5]:
ele.def_integration(3)  #integration and shapefunciton definitions are arbitrary for prototyping.
ele.def_shapefunctions()#should these be done automatically with some 'default' values? how?

J=ele.get_jacobians_isop(ele.intpoints,[0,1])
detJ=ele.get_detj_isop(ele.intpoints,[0,1])
dN_ref=ele.dform0_ref(ele.intpoints)

dN=np.linalg.solve(J,dN_ref)
rho=ele.evaluate_function(rho_sc,ele.intpoints)

AttributeError: 'Triangles_3node' object has no attribute 'def_shapefunctions'

The stiffness matrix:

In [ ]:
K=ele.assemble_gnode(
    ele.integrate_ref(
        rho*np.matmul(dN.transpose((0,1,3,2)),dN)*detJ
    )
)

Then we impose boundary conditions. If
$$(Ka)_i=\sum_{j=1}^n K_{ij}a_j=\sum_{j\in U}K_{ij}a_j+\sum_{j\in B}K_{ij}a_j = 0,$$
where $a_j$ are known boundary values whence $j\in B \subset \{1,...,n\}$, we get
$$ \sum_{j\in U}K_{ij}a_j = - \sum_{j\in B}K_{ij}a_j$$

Defining $S_{ij}=K_{ij}$, if $i,j \in U$ and $S_{ij}=\delta_{ij}$ otherwise and $T_{ij}=K_{ij}$, if $i,j \in B$, and $T_{ij}=0$ otherwise, the solution $a$ may be obtained by solving
$$ Sa=-Tb,$$ where $b_i=a_i$, if $i\in B$ and arbitrary otherwise. 

Now we define the "source vector" determined by the boundary values.

In [ ]:
#the following is way too tedious. could part of this be written into msh-class in some non-trivial-enough way?
#should we think about both linear and nonlinear cases for that? different formulations obviously.
south_nodes=south.gnop.flatten()
north_nodes=north.gnop.flatten()

nodenumber=msh.nodes.shape[0]
b=np.zeros((nodenumber))
b[south_nodes]=u1
b[north_nodes]=u2

s_and_n=np.unique(np.array([south_nodes,north_nodes]).flatten())
allnodes=np.linspace(0,nodenumber-1,nodenumber)
not_sn=np.setdiff1d(allnodes,s_and_n)

In [ ]:
S=sps.eye(K.shape[0]).tocsr()
S[np.ix_(not_sn,not_sn)]=K[np.ix_(not_sn,not_sn)]
T=sps.csr_matrix(K.shape).tolil()
T[np.ix_(s_and_n,s_and_n)]=K[np.ix_(s_and_n,s_and_n)]

In [ ]:
a=sps.linalg.spsolve(S,b-T.dot(b))

In [ ]:
a!=0